In [1]:
import os

%pwd

'/home/tousside/Documents/recrutement/cowrywise-customer-plan-abandonment/research'

os.chdir("../")

%pwd



'/home/ubuntu/africlimateai/rainfall-prediction'

In [2]:
import pandas as pd

data = pd.read_csv("artifacts/data_transformation/train.csv")

data.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   time               187 non-null    object 
 1   temperature        168 non-null    float64
 2   feels_like         168 non-null    float64
 3   app_temp           168 non-null    float64
 4   dew_point          168 non-null    float64
 5   humidity           168 non-null    float64
 6   wind_direction     168 non-null    float64
 7   wind_speed         168 non-null    float64
 8   wind_gust          168 non-null    float64
 9   pressure_relative  168 non-null    float64
 10  pressure_absolute  168 non-null    float64
 11  rainfall           187 non-null    float64
dtypes: float64(11), object(1)
memory usage: 17.7+ KB


In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    model_params: dict
    target_column: str

In [45]:


from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        model_params = self.params.XGBoost
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            model_params = model_params,
            target_column = schema.name
            
        )

        return model_trainer_config

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier  # or any other model
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.linear_model import LogisticRegression
import joblib

In [ ]:

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    def array_to_str_keyed_dict_list(self, X):
        """
        Converts a 2D array into a list of dictionaries
        where keys are string representations of column indices.
        """
        return [dict(zip(map(str, range(X.shape[1])), row)) for row in X]

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]
        
        # gender_id , and type as categorical variables
        categorical_cols = ["gender_id", "type"]
        train_x[categorical_cols] = train_x[categorical_cols].astype(str)
        
        numeric_cols = train_x.select_dtypes(include=["int64", "float64"]).columns.tolist()
        
        # Step 3: ColumnTransformer for imputing
        preprocessor = ColumnTransformer(transformers=[
            ("num", SimpleImputer(strategy="median"), numeric_cols),
            ("scaler", StandardScaler(), numeric_cols),
            ("cat", SimpleImputer(strategy="most_frequent"), categorical_cols)
        ], remainder="passthrough")  # Keep other columns (if any)

        pipeline = Pipeline([
            ("preprocessing", preprocessor),
            ("to_dict", FunctionTransformer(self.array_to_str_keyed_dict_list)),
            ("vectorize", DictVectorizer()),
            ('classifier', XGBClassifier(**self.config.model_params, random_state=42))
        ])

        
        pipeline.fit(train_x, train_y)

        joblib.dump(pipeline, os.path.join(self.config.root_dir, self.config.model_name))

try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

